In [1]:
import pandas as pd
from bokeh.io import curdoc, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Select
from bokeh.plotting import figure
from bokeh.layouts import row, widgetbox

In [2]:
deaths_url = "https://raw.githubusercontent.com/datasets/covid-19/master/data/countries-aggregated.csv"
ds = pd.read_csv(deaths_url)

In [25]:
ds["Date"] = pd.to_datetime(ds["Date"])
countries = ds.Country.values
ds.head()

,Date,Country,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,0,0,0
1,2020-01-22,Albania,0,0,0
2,2020-01-22,Algeria,0,0,0
3,2020-01-22,Andorra,0,0,0
4,2020-01-22,Angola,0,0,0


In [29]:
import numpy as np
countries = np.unique(countries).tolist()

In [30]:
def subset_source(country):
    mask = ds['Country'] == country
    ds_sub = ds[mask].reset_index()
    
    source = ColumnDataSource(data={
        'date' : ds_sub.Date,
        'country' : ds_sub.Country,
        'confirmed' : ds_sub.Confirmed,
        'recovered' : ds_sub.Recovered,
        'dead' : ds_sub.Deaths
    })
    
    return source, ds_sub

In [31]:
selectedCountry = "India"

source, ds_sub = subset_source(selectedCountry)

plot = figure(title = selectedCountry, x_axis_label = 'Dates', y_axis_label = 'Cases Count', x_axis_type = 'datetime', plot_width=700, plot_height=700)

output_file('graph.html')

select = Select(title="Country : ", value="India", options=countries)

plot.line(x = 'date', y = 'confirmed', source = source, color = 'blue')
plot.line(x = 'date', y = 'recovered', source = source, color = 'green')
plot.line(x = 'date', y = 'dead', source = source, color = 'red')

GlyphRenderer(id='1417', ...)

In [6]:
hover = HoverTool(tooltips = [('Confirmed', '@confirmed'), ('Recovered', '@recovered'), ('Deceased', '@dead')])
plot.add_tools(hover)

In [7]:
layout = row(widgetbox(select), plot)

In [8]:
show(layout)

In [19]:
countries

array(['India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'India', 'India', 'India', 'India', 'India', 'India',
       'India', 'Ind